<a href="https://colab.research.google.com/github/girijeshcse/car_finder/blob/sushweta/notebooks/Car_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAPSTONE PROJECT

Our training script begins with a selection of imports. These include:

config: The configuration file consisting of paths and hyperparameters

VGG16: The CNN architecture to serve as the base network for our fine tuning approach

tf.keras: Imports from TensorFlow/Keras consisting of layer types, optimizers, and image loading/preprocessing routines

train_test_split: Scikit-learn’s convenience utility for slicing our network into training and testing subsets

matplotlib: Python’s de facto plotting package

numpy: Python’s standard numerical processing library

cv2: OpenCV

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical


In [ ]:
#import Car_model_config

In [2]:
BASE_PATH = "/content/drive/My Drive/Capstone Data"
TRAIN_IMAGES_PATH = os.path.sep.join([BASE_PATH, "Car Images/Train Images"])
TEST_IMAGES_PATH = os.path.sep.join([BASE_PATH, "Car Images/Test Images"])
TRAIN_ANNOTS_PATH = os.path.sep.join([BASE_PATH, "Annotations/Train Annotations.csv"])
TEST_ANNOTS_PATH = os.path.sep.join([BASE_PATH, "Annotations/Test Annotations.csv"])
CAR_NAMES_MAKE_DICT = os.path.sep.join([BASE_PATH, "Car names and make.csv"])

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# load the contents of the CSV annotations file

# initialize the list of data (images), our target output predictions
# (bounding box coordinates), along with the filenames of the
# individual images
data = []
targets = []
trainfilenames = []
testfilenames = []
labels = []
bboxes = []
trainimagePaths = []



In [5]:
import pandas as pd 
car_map = pd.read_csv(CAR_NAMES_MAKE_DICT)
car_map=car_map.set_index('Cars')


In [6]:
car_map.head()

,Class Numbers
Cars,
AM General Hummer SUV 2000,0
Acura RL Sedan 2012,1
Acura TL Sedan 2012,2
Acura TL Type-S 2008,3
Acura TSX Sedan 2012,4


In [ ]:
#Notuseful

#def add_one(x):
#	return x + 1

#car_map["Class Numbers"] = car_map["Class Numbers"].apply(add_one)
#car_map.head()

In [7]:
car_map_dict=car_map.to_dict('index')
list(car_map_dict.items())[:5]

[('AM General Hummer SUV 2000', {'Class Numbers': 0}),
 ('Acura RL Sedan 2012', {'Class Numbers': 1}),
 ('Acura TL Sedan 2012', {'Class Numbers': 2}),
 ('Acura TL Type-S 2008', {'Class Numbers': 3}),
 ('Acura TSX Sedan 2012', {'Class Numbers': 4})]

In [8]:
#fetch a key based on value passed - we need this to build our image paths
print(list(car_map_dict.keys())[173])

Ram C-V Cargo Van Minivan 2012


In [ ]:

import csv
with open(TRAIN_ANNOTS_PATH, newline='') as file:
    
    reader = csv.reader(file, delimiter = ' ')
      
    # store the headers in a separate variable,
    # move the reader object to point on the next row
    headings = next(reader)
      
    # output list to store all rows
    Output = []
    for row in reader:
        Output.append(row[:])
  
for row in Output:
    row = str(row)
    row = row.split(",")
    (filename, startX, startY, endX, endY, label) = row
    filename = filename.split("'",1)
    fname = filename[1]
    label = label.split("'", 1)
    lbl = int(label[0]) -1
    keyName = list(car_map_dict.keys())[lbl]
    imagePathtrain = os.path.sep.join([TRAIN_IMAGES_PATH, keyName, fname])

    image = cv2.imread(imagePathtrain)
    (h, w) = image.shape[:2]
		# scale the bounding box coordinates relative to the spatial
		# dimensions of the input image
    startX = float(startX) / w
    startY = float(startY) / h
    endX = float(endX) / w
    endY = float(endY) / h
		# load the image and preprocess it
    image = load_img(imagePathtrain, target_size=(224, 224))
    image = img_to_array(image)
		# update our list of data, targets, and filenames

    data = np.append(data, image)
    labels = np.append(labels, lbl)
    bboxes = np.append(bboxes, (startX, startY, endX, endY))
    trainimagePaths = np.append(trainimagePaths,imagePathtrain)

In [ ]:

 
# convert the data, class labels, bounding boxes, and image paths to
# NumPy arrays, scaling the input pixel intensities from the range
# [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
imagePaths = np.array(imagePaths)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# only there are only two labels in the dataset, then we need to use
# Keras/TensorFlow's utility function as well
if len(lb.classes_) == 2:
	labels = to_categorical(labels)

ValueError: ignored

In [ ]:
####to do......
# derive the path to the input image, load the image (in OpenCV
# format), and grab its dimensions
imagePathtest = os.path.sep.join([config.IMAGES_PATH, filename])
image = cv2.imread(imagePathtest)
(h, w) = image.shape[:2]
# scale the bounding box coordinates relative to the spatial
# dimensions of the input image
startX = float(startX) / w
startY = float(startY) / h
endX = float(endX) / w
endY = float(endY) / h

In [ ]:
####to do......
# load the image and preprocess it
image = load_img(imagePathtest, target_size=(224, 224))
image = img_to_array(image)
# update our list of data, targets, and filenames
data.append(image)
targets.append((startX, startY, endX, endY))
testfilenames.append(filename)
 
# convert the data and targets to NumPy arrays, scaling the input
# pixel intensities from the range [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")

In [ ]:
####to do......
# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print("[INFO] saving testing filenames...")
f = open(config.TEST_FILENAMES, "w")
f.write("\n".join(testfilenames))
f.close()

In [ ]:
####to do......
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)

In [ ]:
####to do......
# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(lr=config.INIT_LR)
model.compile(loss="mse", optimizer=opt)
print(model.summary())
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
H = model.fit(
	trainImages, trainTargets,
	validation_data=(testImages, testTargets),
	batch_size=config.BATCH_SIZE,
	epochs=config.NUM_EPOCHS,
	verbose=1)